In [30]:
import pandas as pd
import numpy as np
import datetime

def load_data(spy_file, vix_file):
    df_spy = pd.read_csv(spy_file, parse_dates=["DateTime"])
    df_spy.sort_values("DateTime", inplace=True)
    df_spy.set_index("DateTime", inplace=True)
    df_spy = df_spy.between_time("09:30", "16:00")

    df_vix = pd.read_csv(vix_file, parse_dates=["DateTime"])
    df_vix.sort_values("DateTime", inplace=True)
    df_vix.set_index("DateTime", inplace=True)
    df_vix = df_vix.between_time("09:30", "16:00")

    start = max(df_spy.index.min(), df_vix.index.min())
    end = min(df_spy.index.max(), df_vix.index.max())
    df_spy = df_spy.loc[(df_spy.index >= start) & (df_spy.index <= end)]
    df_vix = df_vix.loc[(df_vix.index >= start) & (df_vix.index <= end)]

    df_vix = df_vix.rename(
        columns={
            "Open": "Open_vix",
            "High": "High_vix",
            "Low":  "Low_vix",
            "Close":"Close_vix"
        }
    )

    df_merged = pd.merge_asof(
        df_spy.sort_index(),
        df_vix.sort_index(),
        left_index=True,
        right_index=True,
        direction='forward'
    )
    return df_merged

def compute_14day_abs_moves(df):
    df["TradeDate"] = df.index.date
    df["TimeOfDay"] = df.index.time
    df["HHMM"] = df.index.strftime("%H:%M")
    open9_30 = df[df["HHMM"]=="09:30"].groupby("TradeDate")["Open"].first().rename("Open9_30")
    df = df.join(open9_30, on="TradeDate")
    df["pct_move_from_open"] = (df["Close"] / df["Open9_30"]) - 1
    df["abs_move"] = df["pct_move_from_open"].abs()
    return df

def rolling_14day_average_move(df):
    df = df.sort_values(["TradeDate","TimeOfDay"]).copy()
    df["avg_move_14d"] = np.nan
    groups = dict(tuple(df.groupby("TimeOfDay")))
    for tod, subdf in groups.items():
        subdf = subdf.sort_values("TradeDate")
        subdf["avg_move_14d"] = subdf["abs_move"].rolling(14, min_periods=14).mean()
        groups[tod] = subdf
    df_final = pd.concat(groups.values(), axis=0)
    df_final = df_final.sort_index()
    return df_final

def compute_noise_boundaries(df):
    df["HHMM"] = df.index.strftime("%H:%M")
    df["PrevClose"] = df.groupby("TradeDate")["Close"].shift(1)
    is_9_30 = (df["HHMM"] == "09:30")
    df.loc[is_9_30, "PrevClose"] = df.loc[is_9_30, "Close"].shift(1)
    df["PrevClose"] = df["PrevClose"].ffill()
    df["RefPriceMax"] = df[["Open9_30","PrevClose"]].max(axis=1)
    df["RefPriceMin"] = df[["Open9_30","PrevClose"]].min(axis=1)
    df["UpperBound"] = df["RefPriceMax"] * (1 + df["avg_move_14d"].fillna(0.0))
    df["LowerBound"] = df["RefPriceMin"] * (1 - df["avg_move_14d"].fillna(0.0))
    return df

def compute_vwap(df):
    df["cum_pv"] = 0.0
    df["cum_vol"] = 0.0

    def calc_cumpv(group):
        return (group["Close"] * group["Volume"]).cumsum()
    def calc_cumvol(group):
        return group["Volume"].cumsum()

    group_obj = df.groupby("TradeDate", group_keys=False)
    cum_pv_series = group_obj.apply(calc_cumpv)
    cum_vol_series = group_obj.apply(calc_cumvol)

    df["cum_pv"] = cum_pv_series
    df["cum_vol"] = cum_vol_series
    df["vwap"] = df["cum_pv"] / df["cum_vol"]
    return df

def compute_daily_volatility(df, n_days=14, target_vol=0.02):
    daily_close = df.groupby("TradeDate")["Close"].last().rename("daily_close")
    daily_ret = daily_close.pct_change().dropna()
    daily_mean = daily_ret.rolling(n_days).mean()
    daily_vol = daily_ret.rolling(n_days).apply(lambda x: np.sqrt(((x - x.mean())**2).mean()))
    vol_df = pd.DataFrame({"Vol": daily_vol, "MeanRet": daily_mean})
    vol_df["Leverage"] = (target_vol / vol_df["Vol"]).clip(upper=4)
    return vol_df

def backtest_strategy(
    df,
    initial_capital=100000.0,
    commission_per_share=0.0035,
    slippage_per_share=0.001,
    target_vol=0.02
):
    df["TimeOfDay"] = df.index.time
    df = df.sort_values(["TradeDate","TimeOfDay"])
    vol_df = compute_daily_volatility(df, n_days=14, target_vol=target_vol)
    df["TradeDate"] = df.index.date
    current_aum = initial_capital
    position = 0
    shares_held = 0
    entry_price = 0.0
    trade_records = []
    last_date = None
    daily_open = df[df.index.strftime("%H:%M")=="09:30"].groupby("TradeDate")["Open"].first()

    def is_semi_hour(t):
        return (t.minute == 0 or t.minute == 30)

    for idx, row in df.iterrows():
        current_date = row["TradeDate"]
        current_time = idx.time()
        current_price = row["Close"]
        upper_bound = row["UpperBound"]
        lower_bound = row["LowerBound"]
        vwap_price = row["vwap"]

        if last_date is None or current_date != last_date:
            open_px = daily_open.get(current_date, np.nan)
            if pd.isnull(open_px):
                open_px = current_price
            leverage = vol_df.loc[current_date, "Leverage"] if current_date in vol_df.index else 1.0
            if pd.isnull(leverage):
                leverage = 1.0
            max_shares_est = (current_aum * leverage) / open_px if open_px>0 else 0
            max_shares = int(np.floor(max_shares_est)) if max_shares_est>0 else 0
            shares_held = 0
            position = 0
            entry_price = 0.0
            last_date = current_date

        if current_time >= datetime.time(16,0):
            if position != 0 and shares_held>0:
                exit_pnl = (current_price - entry_price)*position*shares_held
                fee = (commission_per_share + slippage_per_share)*shares_held
                realized = exit_pnl - fee
                current_aum += realized
                trade_records.append({"date": current_date, "pnl": realized})
            position = 0
            shares_held = 0
            entry_price = 0.0
            continue

        if is_semi_hour(current_time):
            if position != 0 and shares_held>0:
                if position>0:
                    stop_line = max(upper_bound, vwap_price)
                    if current_price <= stop_line:
                        exit_pnl = (current_price - entry_price)*shares_held
                        fee = (commission_per_share + slippage_per_share)*shares_held
                        realized = exit_pnl - fee
                        current_aum += realized
                        trade_records.append({"date": current_date, "pnl": realized})
                        position = 0
                        shares_held = 0
                        entry_price = 0.0
                else:
                    stop_line = min(lower_bound, vwap_price)
                    if current_price >= stop_line:
                        exit_pnl = (entry_price - current_price)*shares_held
                        fee = (commission_per_share + slippage_per_share)*shares_held
                        realized = exit_pnl - fee
                        current_aum += realized
                        trade_records.append({"date": current_date, "pnl": realized})
                        position = 0
                        shares_held = 0
                        entry_price = 0.0

            if position == 0:
                if current_price>upper_bound and max_shares>0:
                    position = 1
                    shares_held = max_shares
                    entry_price = current_price
                elif current_price<lower_bound and max_shares>0:
                    position = -1
                    shares_held = max_shares
                    entry_price = current_price
            else:
                if position>0 and current_price<lower_bound and max_shares>0:
                    exit_pnl = (current_price - entry_price)*shares_held
                    fee = (commission_per_share + slippage_per_share)*shares_held
                    realized = exit_pnl - fee
                    current_aum += realized
                    trade_records.append({"date": current_date, "pnl": realized})
                    position = -1
                    shares_held = max_shares
                    entry_price = current_price
                elif position<0 and current_price>upper_bound and max_shares>0:
                    exit_pnl = (entry_price - current_price)*shares_held
                    fee = (commission_per_share + slippage_per_share)*shares_held
                    realized = exit_pnl - fee
                    current_aum += realized
                    trade_records.append({"date": current_date, "pnl": realized})
                    position = 1
                    shares_held = max_shares
                    entry_price = current_price

    trade_df = pd.DataFrame(trade_records)
    total_pnl = trade_df["pnl"].sum() if not trade_df.empty else 0.0
    trades_count = len(trade_df)
    avg_pnl = total_pnl / trades_count if trades_count else 0.0

    # 按月、年统计
    if not trade_df.empty:
        trade_df["date"] = pd.to_datetime(trade_df["date"])
        trade_df["year"] = trade_df["date"].dt.year
        trade_df["month"] = trade_df["date"].dt.month

        monthly_pnl = trade_df.groupby(["year","month"])["pnl"].sum()
        yearly_pnl = trade_df.groupby(["year"])["pnl"].sum()
    else:
        monthly_pnl = pd.Series(dtype=float)
        yearly_pnl = pd.Series(dtype=float)

    return {
        "FinalAUM": current_aum,
        "TotalPnL": total_pnl,
        "TradesCount": trades_count,
        "AvgPnL": avg_pnl,
        "MonthlyPnL": monthly_pnl,
        "YearlyPnL": yearly_pnl
    }

def main():
    spy_csv = "spy_all.csv"
    vix_csv = "vix_all.csv"
    df_merged = load_data(spy_csv, vix_csv)
    df_merged = compute_14day_abs_moves(df_merged)
    df_merged = rolling_14day_average_move(df_merged)
    df_merged = compute_noise_boundaries(df_merged)
    df_merged = compute_vwap(df_merged)
    result = backtest_strategy(df_merged)
    print("回测结果:")
    print("最终AUM:", result["FinalAUM"])
    print("总盈亏:", result["TotalPnL"])
    print("交易次数:", result["TradesCount"])
    print("平均每笔盈亏:", result["AvgPnL"])
    print("===========")
    print("按月统计:")
    print(result["MonthlyPnL"])
    print("===========")
    print("按年统计:")
    print(result["YearlyPnL"])

if __name__ == "__main__":
    main()

/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_94480/468042207.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cum_pv_series = group_obj.apply(calc_cumpv)
/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_94480/468042207.py:84: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cum_vol_series = group_obj.apply(calc_cumvol)


回测结果:
最终AUM: 113980.1041000003
总盈亏: 13980.104100000535
交易次数: 949
平均每笔盈亏: 14.731405795574853
按月统计:
year  month
2008  1       -6166.4313
      2         232.9750
      3       -1891.6650
      4          64.3440
      5       -1254.8095
                 ...    
2024  11        655.6155
      12        593.0170
2025  1        2359.3455
      2       -5636.3810
      3        2656.3925
Name: pnl, Length: 205, dtype: float64
按年统计:
year
2008   -14767.7933
2009    10342.5060
2010     6372.0098
2011      -46.3863
2012     1506.2924
2013    -2759.7031
2014    -4750.9943
2015     6609.3904
2016     1489.1960
2017     8547.4145
2018    -2523.2111
2019     3560.1485
2020   -10065.3925
2021     5045.7379
2022     4572.6659
2023     3840.6195
2024    -2371.7532
2025     -620.6430
Name: pnl, dtype: float64


In [1]:
import pandas as pd
import numpy as np
from math import floor, sqrt

# -------------------------------
# 辅助函数：计算每日VWAP
# -------------------------------
def compute_vwap(df):
    """计算当日逐分钟累计的VWAP"""
    df = df.copy()
    df['cum_vol'] = df['Volume'].cumsum()
    df['cum_pv'] = (df['Close'] * df['Volume']).cumsum()
    df['VWAP'] = df['cum_pv'] / df['cum_vol']
    return df

# -------------------------------
# 辅助函数：模拟单个交易日内的交易
# -------------------------------
def simulate_day(day_df, allowed_times, position_size):
    """
    模拟单个交易日内的交易（基于Noise Area信号与VWAP动态止损）。
    
    参数：
      day_df: 当日所有分钟数据，需要包含字段 DateTime, Time, Close, UpperBound, LowerBound, VWAP
      allowed_times: 允许触发交易信号的时间点列表，例如 ['09:30', '10:00', ...]
      position_size: 当日固定的交易股数
      
    返回：
      trades: 当日所有交易记录（字典列表），每笔记录包含 entry_time, exit_time, side, entry_price, exit_price, pnl
    """
    position = 0  # 0 表示空仓，1 表示多仓，-1 表示空仓
    entry_price = np.nan
    trailing_stop = np.nan
    trade_entry_time = None
    trades = []
    
    for idx, row in day_df.iterrows():
        current_time = row['Time']
        price = row['Close']
        upper = row['UpperBound']
        lower = row['LowerBound']
        vwap = row['VWAP']
        
        # 在允许的时间点触发开仓信号
        if position == 0 and current_time in allowed_times:
            if price > upper:
                position = 1
                entry_price = price
                trade_entry_time = row['DateTime']
                trailing_stop = max(upper, vwap)  # 多仓初始止损
            elif price < lower:
                position = -1
                entry_price = price
                trade_entry_time = row['DateTime']
                trailing_stop = min(lower, vwap)  # 空仓初始止损
        
        # 如果已有持仓，则更新VWAP和动态止损，并判断是否平仓
        if position != 0:
            if position == 1:
                new_stop = max(upper, vwap)
                trailing_stop = max(trailing_stop, new_stop)
                if price < trailing_stop:
                    exit_time = row['DateTime']
                    pnl = position_size * (price - entry_price)  # 多仓盈亏
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Long',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
            elif position == -1:
                new_stop = min(lower, vwap)
                trailing_stop = min(trailing_stop, new_stop)
                if price > trailing_stop:
                    exit_time = row['DateTime']
                    pnl = position_size * (entry_price - price)  # 空仓盈亏
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Short',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
    # 若日末仍然持仓，则以最后一分钟价格平仓
    if position != 0:
        exit_time = day_df.iloc[-1]['DateTime']
        last_price = day_df.iloc[-1]['Close']
        if position == 1:
            pnl = position_size * (last_price - entry_price)
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Long',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
        else:
            pnl = position_size * (entry_price - last_price)
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Short',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
    return trades

# -------------------------------
# 主程序
# -------------------------------
if __name__ == '__main__':
    # 1. 读取SPY数据 (spy_all.csv)，字段: DateTime, Open, High, Low, Close, Volume
    spy_df = pd.read_csv('spy_all.csv', parse_dates=['DateTime'])
    spy_df.sort_values('DateTime', inplace=True)
    spy_df['Date'] = spy_df['DateTime'].dt.date
    spy_df['Time'] = spy_df['DateTime'].dt.strftime('%H:%M')
    
    # 每日开盘价，取每天第一笔数据
    spy_df['Open_day'] = spy_df.groupby('Date')['Open'].transform('first')
    # 计算每分钟相对于当日开盘价的百分比变化
    spy_df['ret'] = spy_df['Close'] / spy_df['Open_day'] - 1
    
    # 2. 计算过去14天同一时刻的平均绝对涨跌幅（σ）
    pivot = spy_df.pivot(index='Date', columns='Time', values='ret').abs()
    sigma = pivot.rolling(window=14, min_periods=1).mean()
    sigma = sigma.stack().reset_index(name='sigma')
    spy_df = pd.merge(spy_df, sigma, on=['Date', 'Time'], how='left')
    
    # 3. 计算每日收盘价，并获取前一交易日收盘价
    daily_close = spy_df.groupby('Date')['Close'].last().to_frame(name='close').reset_index()
    daily_close['prev_close'] = daily_close['close'].shift(1)
    # 合并到数据中，若首日无prev_close则用当日开盘价替代
    spy_df = pd.merge(spy_df, daily_close[['Date', 'prev_close']], on='Date', how='left')
    spy_df['prev_close'].fillna(spy_df['Open_day'], inplace=True)
    
    # 4. 计算参考值与Noise Area边界
    # 参考上界： max(Open_day, prev_close)，下界： min(Open_day, prev_close)
    spy_df['ref_upper'] = spy_df[['Open_day', 'prev_close']].max(axis=1)
    spy_df['ref_lower'] = spy_df[['Open_day', 'prev_close']].min(axis=1)
    spy_df['UpperBound'] = spy_df['ref_upper'] * (1 + spy_df['sigma'])
    spy_df['LowerBound'] = spy_df['ref_lower'] * (1 - spy_df['sigma'])
    
    # 5. 计算VWAP（基于Volume累加），按日期分组
    spy_df = spy_df.groupby('Date', group_keys=False).apply(compute_vwap)
    
    # 6. 读取VIX数据 (vix_all.csv)，字段: DateTime, Open, High, Low, Close, Volume
    vix_df = pd.read_csv('vix_all.csv', parse_dates=['DateTime'])
    vix_df.sort_values('DateTime', inplace=True)
    vix_df['Date'] = vix_df['DateTime'].dt.date
    vix_df['Time'] = vix_df['DateTime'].dt.strftime('%H:%M')
    vix_min_date = vix_df['Date'].min()
    print("VIX数据最早可用日期：", vix_min_date)
    
    # 7. 动态仓位调整——基于过去14天SPY日收益率标准差
    daily_stats = spy_df.groupby('Date').agg({'Open_day':'first','Close':'last'}).reset_index()
    daily_stats['ret'] = daily_stats['Close'] / daily_stats['Open_day'] - 1
    daily_stats['sigma_spy'] = daily_stats['ret'].rolling(window=14, min_periods=1).std()
    daily_stats.set_index('Date', inplace=True)
    
    # 8. 限制回测范围：仅回测最近5年的数据
    all_dates = sorted(spy_df['Date'].unique())
    last_date = pd.to_datetime(max(all_dates))
    threshold_date = (last_date - pd.DateOffset(years=5)).date()
    # 筛选日期 >= threshold_date 且 >= VIX 最早可用日期
    unique_dates = [d for d in all_dates if d >= threshold_date and d >= vix_min_date]
    
    # 允许交易信号触发的时间点（例如半小时一次）
    allowed_times = ['09:30', '10:00', '10:30', '11:00', '11:30', 
                     '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30']
    
    capital = 100000.0
    daily_results = []  # 用于记录每日资金和收益率
    all_trades = []     # 用于存储所有交易记录
    
    # 开始日内回测：遍历每个交易日，并打印当天的交易记录
    for trade_date in unique_dates:
        day_spy = spy_df[spy_df['Date'] == trade_date].copy().reset_index(drop=True)
        if day_spy.empty:
            continue
        
        # 从VIX数据中获取当天记录，若09:30前无数据则取当天第一条记录
        day_vix = vix_df[vix_df['Date'] == trade_date]
        pre_open_vix = day_vix[day_vix['Time'] < '09:30']
        if pre_open_vix.empty and not day_vix.empty:
            pre_open_vix = day_vix.iloc[[0]]
        if pre_open_vix.empty:
            continue
        vix_value = pre_open_vix.iloc[-1]['Close']
        # 假设VIX为百分比，转换为日化波动率
        daily_vix = (vix_value/100) / sqrt(252)
        
        # 利用SPY数据中每日收益率标准差，如不足则取0.02作为默认值
        sigma_spy = daily_stats.loc[trade_date, 'sigma_spy'] if trade_date in daily_stats.index and daily_stats.loc[trade_date, 'sigma_spy']>0 else 0.02
        target_vol = 0.02
        multiplier = min(4, target_vol / sigma_spy)
        
        open_price = day_spy.iloc[0]['Open_day']
        position_size = floor(capital * multiplier / open_price)
        if position_size <= 0:
            continue
        
        # 模拟当天交易，得到交易记录
        trades = simulate_day(day_spy, allowed_times, position_size)
        # 打印当天的交易记录
        print(f"\n日期: {trade_date}  交易记录:")
        if trades:
            for t in trades:
                print(t)
        else:
            print("  无有效交易记录")
        
        day_pnl = sum(trade['pnl'] for trade in trades)
        capital_start = capital
        capital = capital + day_pnl
        daily_return = (capital - capital_start) / capital_start
        daily_results.append({'Date': trade_date, 'capital': capital, 'daily_return': daily_return})
        for t in trades:
            t['date'] = trade_date
            all_trades.append(t)
    
    # 汇总每日结果，计算每月和每年累计收益率
    daily_df = pd.DataFrame(daily_results)
    daily_df['Date'] = pd.to_datetime(daily_df['Date'])
    daily_df.sort_values('Date', inplace=True)
    daily_df.set_index('Date', inplace=True)
    
    monthly = daily_df.resample('M').agg({'capital': ['first', 'last']})
    monthly.columns = ['month_start', 'month_end']
    monthly['monthly_return'] = monthly['month_end'] / monthly['month_start'] - 1
    
    yearly = daily_df.resample('A').agg({'capital': ['first', 'last']})
    yearly.columns = ['year_start', 'year_end']
    yearly['yearly_return'] = yearly['year_end'] / yearly['year_start'] - 1
    
    print("\n每个月累计收益率:")
    print(monthly[['month_start','month_end','monthly_return']])
    
    print("\n每年累计收益率:")
    print(yearly[['year_start','year_end','yearly_return']])
    
    total_return = capital / 100000.0 - 1
    print(f"\n整个回测期间累计收益率: {total_return*100:.2f}%")


/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_12012/3691695736.py:143: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  spy_df['prev_close'].fillna(spy_df['Open_day'], inplace=True)
/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_12012/3691695736.py:153: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the grouping

VIX数据最早可用日期： 2008-01-02

日期: 2020-04-06  交易记录:
  无有效交易记录

日期: 2020-04-07  交易记录:
  无有效交易记录

日期: 2020-04-08  交易记录:
{'entry_time': Timestamp('2020-04-08 11:30:00'), 'exit_time': Timestamp('2020-04-08 12:00:00'), 'side': 'Long', 'entry_price': 270.59, 'exit_price': 270.57, 'pnl': -3.7199999999966167}
{'entry_time': Timestamp('2020-04-08 14:30:00'), 'exit_time': Timestamp('2020-04-08 15:40:00'), 'side': 'Long', 'entry_price': 272.87, 'exit_price': 272.42, 'pnl': -83.69999999999789}

日期: 2020-04-09  交易记录:
  无有效交易记录

日期: 2020-04-13  交易记录:
  无有效交易记录

日期: 2020-04-14  交易记录:
{'entry_time': Timestamp('2020-04-14 10:30:00'), 'exit_time': Timestamp('2020-04-14 10:32:00'), 'side': 'Long', 'entry_price': 282.78, 'exit_price': 282.57, 'pnl': -47.039999999995416}

日期: 2020-04-15  交易记录:
{'entry_time': Timestamp('2020-04-15 10:30:00'), 'exit_time': Timestamp('2020-04-15 10:45:00'), 'side': 'Short', 'entry_price': 276.12, 'exit_price': 276.615, 'pnl': -107.91000000000099}
{'entry_time': Timestamp('2020-04-

/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_12012/3691695736.py:236: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = daily_df.resample('M').agg({'capital': ['first', 'last']})
/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_12012/3691695736.py:240: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly = daily_df.resample('A').agg({'capital': ['first', 'last']})
